In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive

# Montamos Google Drive
drive.mount('/content/drive')

# Define la URL base y las fechas de inicio y fin
url_base = 'https://elcomercio.pe/archivo/todas/'
fecha_inicio = '2023-08-31'
fecha_fin = '2023-09-30'

# Lista para almacenar los DataFrames de cada fecha
dfs = []

# Carpeta de destino para guardar las imágenes
image_folder = '/content/drive/My Drive/scrapy/Imagenes/'

# Recorre las fechas desde fecha_inicio hasta fecha_fin
current_fecha = fecha_inicio
while current_fecha <= fecha_fin:
    # Construye la URL con la fecha actual
    url = f'{url_base}{current_fecha}/'

    # Realizamos una solicitud GET a la página web
    response = requests.get(url)

    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        content = response.text

        # Parsear el contenido HTML utilizando BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Encontrar todas las imágenes en la página
        img_tags = soup.find_all('img', class_='story-item__img')  # Cambia 'img' por el selector adecuado para tu página web

        # Lista para almacenar las rutas de las imágenes
        image_paths = []

        # Descargar las imágenes y guardar sus rutas
        for img_tag in img_tags:
            img_url = img_tag.get('src')
            if img_url and img_url.startswith('http'):
                img_response = requests.get(img_url)
                if img_response.status_code == 200:
                    img_filename = f'{current_fecha}_{img_url.split("/")[-1]}'  # Nombre de archivo único
                    img_path = f'{image_folder}{img_filename}'
                    with open(img_path, 'wb') as img_file:
                        img_file.write(img_response.content)
                    image_paths.append(img_path)

        # Crear un DataFrame de pandas con la fecha y las rutas de las imágenes
        df = pd.DataFrame({'Fecha': [current_fecha] * len(image_paths), 'Ruta de Imagen': image_paths})

        # Agregar el DataFrame a la lista
        dfs.append(df)
    else:
        print(f"No se pudo acceder a la página web para la fecha {current_fecha}.")

    # Avanzar a la siguiente fecha
    current_fecha = pd.to_datetime(current_fecha) + pd.DateOffset(days=1)
    current_fecha = current_fecha.strftime('%Y-%m-%d')

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)

# Especifica la ruta completa del archivo CSV en tu Google Drive
csv_filepath = '/content/drive/My Drive/scrapy/ScrapingJPG.csv'

# Guardar el DataFrame en un archivo CSV en Google Drive
final_df.to_csv(csv_filepath, index=False)

print(f"Se han descargado y guardado las imágenes junto con las fechas en '{csv_filepath}'.")
print(len(final_df))

Mounted at /content/drive
Se han descargado y guardado las imágenes junto con las fechas en '/content/drive/My Drive/scrapy/ScrapingJPG.csv'.
3100
